In [1]:
import os
import glob
import pydicom
import nibabel as nib
import pandas as pd
import numpy as np
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2

%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from utils.dcm_utils import *
from utils.nii_utils import *

from PIL import Image

import csv
from tqdm import tqdm

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
DATA_DIR = "/Volumes/SSD970/"
TRAIN_IMAGE_DIR = os.path.join(DATA_DIR, "train_images")
XRAY_IMAGE_DIR = os.path.join(DATA_DIR, "xray_images")

train_df = pd.read_csv(os.path.join(DATA_DIR, "train_df_with_slices.csv"))

In [3]:
patient_dirs = glob.glob(TRAIN_IMAGE_DIR + "/**")
len(patient_dirs)

2012

In [4]:
len(patient_dirs)
header = [
    'UID', 'direction',
    'z_spacing', 'pixel_spacing', 'aspect',
    'width', 'height'
]

In [5]:
f = open('meta_xray.csv','w',newline='')
writer = csv.writer(f)
writer.writerow(header)

59

In [8]:
def save_xray_image(UID, patient_train_dir):
    patient_xray_dir = os.path.join(XRAY_IMAGE_DIR, UID)

    if os.path.exists(os.path.join(patient_xray_dir, "axial.jpeg")):
        return False

    if os.path.exists(patient_xray_dir) == False:
        os.mkdir(patient_xray_dir)

    imgs, image_positions, image_orientations, pixel_spacings, slice_thicknesses, is_flip = read_patient_dcm(patient_train_dir)

    z_spacing = image_positions[0, 2] - image_positions[1, 2]
    pixel_spacing = pixel_spacings[0, 0]
    aspect = z_spacing / pixel_spacing

    # print(imgs.shape)
    imgs = np.clip(imgs, a_min=0, a_max=2242)
    # imgs = normalize_hu(imgs)

    axial = np.mean(imgs, axis=0)
    axial = Image.fromarray(np.uint8(axial / axial.max() * 255), 'L')

    sagittal = np.mean(imgs, axis=2)
    sagittal = Image.fromarray(np.uint8(sagittal / sagittal.max() * 255), 'L')

    coronal = np.mean(imgs, axis=1)
    coronal = Image.fromarray(np.uint8(coronal / coronal.max() * 255), 'L')

    axial.save(os.path.join(patient_xray_dir, "axial.jpeg"))

    z_height = int(imgs.shape[0] * aspect)

    sagittal_size = [imgs.shape[1], z_height]
    sagittal = sagittal.resize(sagittal_size)
    sagittal.save(os.path.join(patient_xray_dir, "sagittal.jpeg"))

    coronal_size = [imgs.shape[2], z_height]
    coronal = coronal.resize(coronal_size)
    coronal.save(os.path.join(patient_xray_dir, "coronal.jpeg"))

    writer.writerows([
        [
            UID, 0,
            z_spacing, pixel_spacing, aspect,
            axial.width, axial.height
        ],
        [
            UID, 1,
            z_spacing, pixel_spacing, aspect,
            sagittal.width, sagittal.height
        ],
        [
            UID, 2,
            z_spacing, pixel_spacing, aspect,
            coronal.width, coronal.height
        ]
    ])

# patient_train_dir = patient_dirs[0]
# UID = patient_train_dir.split("/")[-1]
# save_xray_image(UID, patient_train_dir)

In [9]:
for patient_dir in tqdm(patient_dirs):
    UID = patient_dir.split("/")[-1]
    save_xray_image(UID, patient_dir)

 69%|██████▉   | 1389/2012 [1:07:14<16:16,  1.57s/it]  /Users/longyikim/anaconda3/envs/pytorch/lib/python3.9/site-packages/openjpeg/utils.py:209: UserWarning: The (0028,0101) Bits Stored value '12' in the dataset does not match the component precision value '16' found in the JPEG 2000 data. It's recommended that you change the Bits Stored value to produce the correct output
  warnings.warn(
100%|██████████| 2012/2012 [1:35:18<00:00,  2.84s/it]


In [ ]:
f.close()